In [1]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')
nltk.download('wordnet')
nltk.download('omw-1.4')
import spacy
from spacy.language import Language
from spacy import displacy
from spacy.tokens import Span
from spacy.matcher import PhraseMatcher
from spacy.matcher import Matcher
from nltk.tokenize import word_tokenize
import re

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/balazsszabo/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/balazsszabo/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /Users/balazsszabo/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     /Users/balazsszabo/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/balazsszabo/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/balazsszabo/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [2]:
@Language.factory('make_street_entity_matcher')
def make_street_entity_matcher(nlp, name, file_name:str, entity_name:str, pattern):
    return EntityMatcher(nlp, file_name, entity_name,pattern)

re.compile('^[A-Z][0-9]{1,5}.*')
^[A-Z][0-9]{1,5}.*


In [6]:
class EntityMatcher:
    def __init__(self, nlp, file_name, entity_name, pattern):
        if file_name:
            line_list = open(file_name,'r').readlines()
            phrase_list = []
            for l in line_list:
                phrase_list.append(l.strip())
            phrase_patterns = [nlp(text) for text in phrase_list]
            self.phrase_matcher = PhraseMatcher(nlp.vocab, attr="LOWER")
            self.phrase_matcher.add(entity_name, None, *phrase_patterns )
        if pattern:
            self.matcher = Matcher(nlp.vocab)
            self.matcher.add(entity_name, [pattern] )

    def __call__(self, doc):
        matches = set()
        if self.matcher:
            matches.update(self.matcher(doc))

        if self.phrase_matcher:
            matches.update(self.phrase_matcher(doc))
        seen_tokens = set()
        new_entities = []
        entities = doc.ents
        for match_id, start, end in matches:
            if start not in seen_tokens and end - 1 not in seen_tokens:
                new_entities.append(Span(doc, start, end, label=match_id))
                entities = [
                    e for e in entities if not (e.start < end and e.end > start)
                ]
                seen_tokens.update(range(start, end))

        doc.ents = tuple(entities) + tuple(new_entities)
        return doc

In [7]:
#nlp = spacy.load("nl_core_news_sm")
nlp = spacy.load("en_core_web_sm")

regex =[{"TEXT": {"REGEX": "^[A-Z][0-9]{1,5}.*"}}]
nlp.add_pipe("make_street_entity_matcher", config={"file_name":'../../Data/street_names_north_holland.txt',"entity_name": "STREET", "pattern": regex}, after="ner")

In [8]:
file = '../../Data/AH/Haarlemmerdijk 1.txt'
tokenisedList = []
with open(file,'r') as f:
    line = f.readline()
    while line:
        line = f.readline()
        nlpLine = nlp(line)
        tokenisedList.append(nlpLine)

In [ ]:
# if you want to keep specific stop words
# nlp.Defaults.stop_words.remove()
# nlp.vocab['beyond'].is_stop = False

In [9]:
colors = {"STREET": '#981231'}
options={'distance':50,'colors': colors}
displacy.render(tokenisedList, style='ent',options=options, jupyter=True)
#displacy.serve(tokenisedList, style='ent', options=options )

/Users/balazsszabo/Desktop/VU-IS/Thesis design/Model/venv/lib/python3.6/site-packages/spacy/displacy/__init__.py:200: UserWarning: [W006] No entities to visualize found in Doc object. If this is surprising to you, make sure the Doc was processed using a model that supports named entity recognition, and check the `doc.ents` property manually if necessary.
  warnings.warn(Warnings.W006)
